In [ ]:
# Install the Kaggle package using pip
!pip install kaggle

In [ ]:
# Import necessary libraries and modules

# NumPy for numerical operations
from numpy import *

# TensorFlow for deep learning
from tensorflow import *

# Pickle for handling data serialization
from pickle import *

# Keras for building and training neural networks
from keras import layers, activations, models, preprocessing, utils

# Gensim for Word2Vec model
from gensim.models import Word2Vec

In [ ]:
# Import the 'files' module from the google.colab library
from google.colab import files

# Use the files.upload() function to prompt the user to upload files
files.upload()
#upload your api of Kaggle .json file

TypeError: ignored

In [ ]:
# Create the .kaggle directory in the user's home directory if it doesn't exist
!mkdir -p ~/.kaggle

In [ ]:
# Copy the Kaggle API credentials file (kaggle.json) to the .kaggle directory
!cp kaggle.json ~/.kaggle/
# Set the correct permissions on the Kaggle API credentials file
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
# Download the Kaggle dataset named "chatterbotenglish" using the Kaggle CLI
!kaggle datasets download -d kausr25/chatterbotenglish

In [ ]:
# Create a directory named "data"
!mkdir data

# Note: The mkdir command is used to create a new directory. In this case, it's named "data" to potentially store the unzipped contents.

# Unzip the contents of the "chatterbotenglish.zip" file into the "data" directory
!unzip /content/chatterbotenglish.zip -d data

# Note: The unzip command is used to extract the contents of a zip file. The "-d" flag specifies the target directory ("data" in this case),
# and "/content/chatterbotenglish.zip" is the path to the zip file. This assumes the zip file is located at that path.
# The contents of the zip file will be extracted into the specified directory.

#choose y[es] for questions


In [ ]:
# Import the 'os' module for interacting with the operating system
import os

# Note: The 'os' module provides a way to interact with the operating system, allowing tasks such as file operations, directory manipulation, etc.

# Import the 'yaml' module for working with YAML files
import yaml

# Note: The 'yaml' module is used for reading and writing YAML (YAML Ain't Markup Language) files.
# YAML is often used for configuration files due to its human-readable and concise syntax.


In [ ]:
# Specify the directory path
path = '/content/data'

# Note: The 'path' variable holds the path to the directory where files will be listed.
# '/content/data' is an example path; you can replace it with the actual path you want to work with.

# Use os.listdir() to get a list of files in the specified directory
files = os.listdir(path + os.sep)


In [ ]:
# Initialize empty lists for questions and answers
que = list()
ans = list()

# Loop through each file in the specified directory
for filepath in files:
    # Open the file in binary mode and load its content using yaml.safe_load
    f = open(path + os.sep + filepath, 'rb')
    fl = yaml.safe_load(f)

    # Extract conversations (Q&A pairs) from the loaded YAML file
    qna = fl['conversations']

    # Process each conversation
    for j in qna:
        if len(j) > 2:
            # If the conversation has more than two elements, consider the first as a question
            # and concatenate the rest as the answer
            que.append(j[0])
            remaining = j[1:]
            a = ''
            for k in remaining:
                a += ' ' + k
            ans.append(a)
        elif len(j) > 1:
            # If the conversation has exactly two elements, consider them as a question and an answer
            que.append(j[0])
            ans.append(j[1])

# Initialize an empty list for checking the format of answers
format_check = list()

# Check the format of answers and populate the format_check list
for i in range(len(ans)):
    if type(ans[i]) == str:
        format_check.append(ans[i])
    else:
        que.pop(i)  # Remove the corresponding question if the answer is not a string

# Reinitialize the ans list and prepend and append start and end tokens to each answer
ans = list()
start_token = '<start>'
end_token = '<end>'
for i in range(len(format_check)):
    ans.append(' ' + format_check[i] + ' ')

# Prepend and append start and end tokens to each answer
ans = [start_token + x + end_token for x in ans]

# Initialize a text tokenizer and fit it on both questions and answers
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(que + ans)

# Determine the vocabulary size
VOCAB_SIZE = len(tokenizer.word_index) + 1

# Note: The above code processes a set of files containing conversations (Q&A pairs) in YAML format.
# It extracts questions and answers, formats the answers, and then tokenizes both questions and answers
# using the Keras Tokenizer. The resulting vocabulary size is determined.


In [ ]:
# Import the 're' module for regular expressions
import re

# Initialize an empty list to store the knowledge extracted from the tokenizer's word index
knowledge = []

# Iterate through each word in the tokenizer's word index and add it to the knowledge list
for i in tokenizer.word_index:
    knowledge.append(i)

# Define a function 'tokenize' that takes a list of sentences 's' as input
def tokenize(s):
    # Initialize empty lists for tokenized sentences (tl) and individual vocabulary words (vc)
    tl = []
    vc = []

    # Process each sentence in the input list
    for j in s:
        # Convert the sentence to lowercase
        j = j.lower()

        # Use regular expression to replace non-alphabetic characters with spaces
        j = re.sub('[^a-zA-Z]', ' ', j)

        # Split the processed sentence into words and add them to the vocabulary list 'vc'
        t = j.split()
        vc += t

        # Append the tokenized sentence to the 'tl' list
        tl.append(t)

    # Return the tokenized sentences and the vocabulary list
    return tl, vc


In [ ]:
# Tokenize the questions and pad the sequences
tk_que = tokenizer.texts_to_sequences(que)
q_max = max([len(i) for i in tk_que])
q_padded = preprocessing.sequence.pad_sequences( tk_que, maxlen = q_max, padding = 'post')
enc_data = array(q_padded)

In [ ]:
# Tokenize the answers and pad the sequences
tk_ans = tokenizer.texts_to_sequences(ans)
a_max = max([len(i) for i in tk_ans])
a_padded = preprocessing.sequence.pad_sequences( tk_ans, maxlen = a_max, padding = 'post')
dec_data = array(a_padded)

In [ ]:
tk_ans = tokenizer.texts_to_sequences(ans)
# Shift the answer sequences to the left to prepare input for the decoder

for i in range(len(tk_ans)):
  tk_ans[i] = tk_ans[i][1:]
a_padded = preprocessing.sequence.pad_sequences( tk_ans, maxlen = a_max, padding = 'post')
answ = utils.to_categorical(a_padded, VOCAB_SIZE)
dec_out = array(answ)

In [ ]:
# Define the encoder part of the model

encoder_inputs = keras.layers.Input(shape=( q_max , ))
encoder_embedding = keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = keras.layers.LSTM( 200 , return_state=True )( encoder_embedding )
encoder_states = [ state_h , state_c ]

# Define the decoder part of the model
decoder_inputs = keras.layers.Input(shape=( a_max ,  ))
decoder_embedding = keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True) (decoder_inputs)
decoder_lstm = keras.layers.LSTM( 200 , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = keras.layers.Dense( VOCAB_SIZE , activation=keras.activations.softmax )
output = decoder_dense ( decoder_outputs )

#Define the keras model:w
model = keras.models.Model([encoder_inputs, decoder_inputs], output )
optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

#Print the summary of the model:
model.summary()

In [ ]:
#
model.fit([enc_data, dec_data], dec_out, batch_size = 30, epochs = 200)
model.save('model.h5')

In [ ]:
def lstm_io():
  encoder_model = models.Model(encoder_inputs, encoder_states)

  decoder_state_input_h = layers.Input(shape=( 200 ,))
  decoder_state_input_c = layers.Input(shape=( 200 ,))

  decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

  decoder_outputs, state_h, state_c = decoder_lstm(decoder_embedding , initial_state=decoder_states_inputs)

  decoder_states = [state_h, state_c]

  decoder_outputs = decoder_dense(decoder_outputs)

  decoder_model = models.Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

  return encoder_model , decoder_model

In [ ]:
def str_to_tokens( sentence : str ):

    words = sentence.lower().split()
    tkl = list()

    for i in words:
        tkl.append(tokenizer.word_index[i])
    return preprocessing.sequence.pad_sequences([tkl], maxlen=q_max, padding='post')

In [ ]:
e_model, d_model = lstm_io()

In [ ]:
import numpy as np
enc_model , dec_model = lstm_io()

for i in range(10):
    states_values = enc_model.predict(str_to_tokens(input('Enter question : ' ) ) )
    empty_target_seq = np.zeros((1 , 1))
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stp = False
    decoded_translation = ''
    while not stp:
        dec_outputs ,h ,c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for  w, i in tokenizer.word_index.items() :
            if sampled_word_index == i:
                decoded_translation += ' {}'.format(w)
                sampled_word = w

        if (sampled_word == 'end' or len(decoded_translation.split()) > a_max):
            stp = True

        empty_target_seq = np.zeros( ( 1 , 1 ) )
        empty_target_seq[0, 0] = sampled_word_index
        states_values = [h, c]

    print( decoded_translation )